# Exercícios - DataFrames com Pandas (Parte 3)

In [277]:
import pandas as pd
import requests

##  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [279]:
# 1) A URL pode ser obtida no link https://apidatalake.tesouro.gov.br/docs/sadipem/#/PVL%20-%20Dados%20Básicos/get_pvl
# Aqui, utilizaremos a URL abaixo
URL = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

request = requests.get(URL)
# status_code = 200 significa que a url está funcional
request.status_code

200

In [280]:
# O arquivo de solicitação é um json
# Usando .json() para acessar seus elementos
request_json = request.json()

In [281]:
# Gerando o DataFrame com os "items" do JSON
df = pd.DataFrame(request_json['items'])
df.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,28387,Município,Rio das Antas,4214409,SC,PVL02.000996/2018-71,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado de Santa Catarina...,Real,1200000.0,0,1,22/05/2018
1,41957,Município,Terra Roxa,4127403,PR,PVL02.000619/2020-56,Arquivado a pedido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2000000.0,0,0,25/03/2020
2,63429,Município,Joinville,4209102,SC,PVL02.000262/2023-59,Deferido,17944.100564/2023-99,2023-09-06T18:02:55Z,Concessão de garantia,Concessão de garantia a empresa estatal não de...,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,128000000.0,0,0,22/09/2023


In [282]:
# No momento o df possui todos os estados
print(df['uf'].unique())

# Filtrando somente RJ
df_RJ = df[df['uf'] == 'RJ']

print(f'--> Agora temos a "uf" = {df_RJ['uf'].unique()}')
df_RJ.head(3)

['SC' 'PR' 'AL' 'SP' 'ES' 'TO' 'RS' 'MG' 'MT' 'GO' 'CE' 'MS' 'RJ' 'BA'
 'RN' 'SE' 'RR' 'PA' 'AM' 'PB' 'PI' 'AC' 'PE' 'RO' 'MA' 'DF' 'AP']
--> Agora temos a "uf" = ['RJ']


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
19,41862,Município,Santa Maria Madalena,3304607,RJ,PVL02.009405/2019-10,Em retificação pelo interessado,17944.100701/2020-42,2020-02-19T16:20:58Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4770000.0,0,0,06/04/2020
22,45530,Município,Casimiro de Abreu,3301306,RJ,PVL02.001056/2021-02,Arquivado a pedido,17944.102661/2021-54,2021-07-30T19:40:33Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,29000000.0,0,0,04/11/2021
60,9898,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.001076/2010-85,2011-05-25T03:00:00Z,Operação contratual interna,Saúde,Instituição Financeira Nacional,Banco do Brasil S/A,Real,132800000.0,0,0,07/06/2011


#### 2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?

In [284]:
# 2) 
status = df_RJ['status'].value_counts()
# Gerando um dataframe os status e suas frequências
df_status = pd.DataFrame(status)
df_status

,count
status,
Deferido,24
Encaminhado à PGFN com manifestação técnica favorável,10
Arquivado,8
Deferido (PVL-IF),4
Regularizado,3
Arquivado por decurso de prazo,3
Em retificação pelo interessado,1
Arquivado a pedido,1
Assinado pelo interessado (retificação),1


In [285]:
# Outra forma seria simplesmentes fazer
df_RJ.status.value_counts()

status
Deferido                                                 24
Encaminhado à PGFN com manifestação técnica favorável    10
Arquivado                                                 8
Deferido (PVL-IF)                                         4
Regularizado                                              3
Arquivado por decurso de prazo                            3
Em retificação pelo interessado                           1
Arquivado a pedido                                        1
Assinado pelo interessado (retificação)                   1
Em retificação pelo credor                                1
Arquivado a pedido (PVL-IF)                               1
Name: count, dtype: int64

#### 3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).

In [287]:
df_RJ.dtypes

id_pleito                  int64
tipo_interessado          object
interessado               object
cod_ibge                   int64
uf                        object
num_pvl                   object
status                    object
num_processo              object
data_protocolo            object
tipo_operacao             object
finalidade                object
tipo_credor               object
credor                    object
moeda                     object
valor                    float64
pvl_assoc_divida           int64
pvl_contradado_credor      int64
data_status               object
dtype: object

In [288]:
# Manipulando e convertendo a coluna 'data_status'
strings = df_RJ['data_status'].str.split('/')
strings.index = range(0, len(strings))
strings.head()

0    [06, 04, 2020]
1    [04, 11, 2021]
2    [07, 06, 2011]
3    [14, 09, 2016]
4    [27, 06, 2024]
Name: data_status, dtype: object

In [289]:
# Obtendo apenas o ano e armazenando na variável
ano_status = []
for day, month, year in strings.values:
    # .values para obtermos apenas as listas, sem index
    ano_status.append(year)
print(ano_status)

['2020', '2021', '2011', '2016', '2024', '2008', '2016', '2007', '2019', '2019', '2019', '2023', '2023', '2002', '2019', '2024', '2013', '2011', '2011', '2015', '2010', '2014', '2011', '2007', '2012', '2008', '2014', '2008', '2009', '2008', '2007', '2006', '2012', '2007', '2015', '2008', '2007', '2009', '2007', '2003', '2014', '2007', '2023', '2022', '2011', '2004', '2014', '2023', '2022', '2017', '2014', '2017', '2008', '2016', '2011', '2023', '2007']


#### 4. Indique a frequência de cada ano do campo construído no item (3).

In [291]:
# 4) Por fins de simplicidade vamos criar uma "Series"
ano_status_series = pd.Series(ano_status)
# Utilizando o método .value_counts() para obter a contagem
ano_status_series.value_counts()

2007    8
2011    6
2008    6
2023    5
2014    5
2019    4
2016    3
2017    2
2024    2
2022    2
2015    2
2012    2
2009    2
2006    1
2004    1
2003    1
2020    1
2010    1
2021    1
2002    1
2013    1
Name: count, dtype: int64

## 2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

#### 1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado ('Estado'ou Município), e que devolve os dados da consulta no formato DataFrame.

In [294]:
def consulta_uf_tipo(uf: str, tipo_de_interessado: str):
    # A request
    data = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl')
    # Criando o DataFrame 
    df = pd.DataFrame(data.json()['items'])
    # Condição/consulta
    consulta = (df['uf'] == uf) & (df['tipo_interessado'] == tipo_de_interessado)
    
    return df[consulta]

In [295]:
# Avaliando a forma do df retornado
print(consulta_uf_tipo('SP', 'Estado').shape)
# O df retornado
consulta_uf_tipo('SP', 'Estado').head()

(26, 18)


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
225,7056,Estado,São Paulo,35,SP,None,Deferido,17944.000487/2013-04,2015-12-16T02:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.500000e+09,1,0,23/09/2013
395,11263,Estado,São Paulo,35,SP,None,Arquivado,17944.001396/2011-16,2011-09-21T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,7.486330e+08,0,0,10/10/2011
444,11493,Estado,São Paulo,35,SP,None,Deferido,17944.001461/2008-16,2008-08-20T03:00:00Z,Concessão de garantia,Concessão de garantia a empresa estatal não de...,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.000000e+08,1,0,27/08/2008
790,67621,Estado,São Paulo,35,SP,PVL02.004327/2023-35,Deferido,17944.000023/2024-42,2024-02-29T16:01:25Z,Operação contratual interna,Aporte em PPP,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.354898e+09,0,0,13/03/2024
839,28058,Estado,São Paulo,35,SP,PVL02.000587/2018-74,Encaminhado à PGFN com manifestação técnica fa...,17944.102744/2018-48,2018-04-25T12:36:06Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Santander (Brasil) S.A.,Real,2.500000e+08,1,1,04/05/2018


In [296]:
#Para os municípios (São Paulo)
print(consulta_uf_tipo('SP', 'Município').shape)
consulta_uf_tipo('SP', 'Município').head()

(620, 18)


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
4,67257,Município,Palestina,3535002,SP,PVL02.004185/2023-14,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2000000.0,0,1,29/11/2023
14,68585,Município,Ibiúna,3519709,SP,PVL02.000900/2024-12,Em análise (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado de São Paulo,Real,2580000.0,0,0,19/06/2024
18,39395,Município,Guaíra,3517406,SP,PVL02.008066/2019-46,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3042242.0,1,1,27/04/2020
20,42483,Município,São Sebastião,3550704,SP,PVL02.005643/2019-48,Deferido,17944.101980/2020-61,2020-04-22T19:31:17Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,99800000.0,1,1,27/04/2020
23,45037,Município,Presidente Venceslau,3541505,SP,PVL02.001248/2021-19,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado de São Paulo,Real,500000.0,0,1,28/09/2021


In [297]:
# Para o estado (Mato Grosso)
print(consulta_uf_tipo('MT', 'Estado').shape)
consulta_uf_tipo('MT','Estado').head()

(5, 18)


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
2419,8569,Estado,Mato Grosso,51,MT,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000800/2014-87,2017-04-13T21:04:55Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,4.700000e+08,1,0,25/04/2017
2781,8505,Estado,Mato Grosso,51,MT,None,Deferido,17944.000788/2010-87,2010-06-11T03:00:00Z,Operação contratual interna,PMAE,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.009964e+07,1,0,18/06/2010
3273,12222,Estado,Mato Grosso,51,MT,None,Deferido,17944.001630/2009-91,2009-12-29T02:00:00Z,Operação contratual interna,PSI - Programa de Sustentação do Investimento,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.388059e+07,1,0,30/12/2009
4152,27000,Estado,Mato Grosso,51,MT,PVL02.002529/2017-02,Deferido,17944.103369/2017-72,2018-02-26T14:03:22Z,Operação contratual interna,Renegociação de dívidas,União,União,Real,1.895747e+09,0,0,05/03/2018
4631,42747,Estado,Mato Grosso,51,MT,PVL02.001213/2020-91,Deferido,17944.102461/2020-11,2020-06-03T23:09:01Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.550000e+08,0,0,05/06/2020


In [298]:
# Para os municípios (MT)
print(consulta_uf_tipo('MT', 'Município').shape)
consulta_uf_tipo('MT','Município').head()

(87, 18)


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
10,65516,Município,Juína,5105150,MT,PVL02.001731/2023-57,Deferido,17944.102872/2023-59,2023-10-30T20:41:47Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,19000000.0,0,0,03/11/2023
56,9183,Município,Barra do Garças,5101803,MT,None,Arquivado,17944.000921/2006-19,2007-10-18T02:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco do Brasil S/A,Real,897300.0,0,0,01/02/2008
131,7697,Município,Várzea Grande,5108402,MT,None,Deferido,17944.000636/2004-36,2004-06-08T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,13056000.0,1,0,25/06/2004
197,10044,Município,Lucas do Rio Verde,5105259,MT,None,Arquivado,17944.001107/2009-64,2010-06-24T03:00:00Z,Operação contratual interna,Caminho da Escola 2009 A,Instituição Financeira Nacional,Caixa Econômica Federal,Real,561600.0,0,0,21/01/2011
250,28121,Município,Rondonópolis,5107602,MT,PVL02.000721/2018-37,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,4965000.0,0,1,18/12/2019


#### 2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo' estão registradas?

In [300]:
# Consultando o estado por meio da função 
MG = consulta_uf_tipo('MG', 'Estado')
# Verificando o status
MG['status'].value_counts()

status
Encaminhado à PGFN com manifestação técnica favorável    3
Em retificação pelo interessado                          1
Arquivado                                                1
Arquivado a pedido                                       1
Name: count, dtype: int64

> **Não há solicitações com o status 'Arquivado por decurso de prazo' para o *Estado* de Minas Gerais.** Como pode ser verificado pelo DataFrame correspondente (abaixo).

In [302]:
# DataFrame MG
MG

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
1024,9197,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000924/2009-03,2010-03-10T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,50000000.0,1,0,01/04/2010
1388,68204,Estado,Minas Gerais,31,MG,PVL02.003861/2023-24,Em retificação pelo interessado,17944.001186/2024-42,2024-03-19T14:59:50Z,Concessão de garantia,Concessão de garantia a empresa estatal não de...,Instituição Financeira Internacional,New Development Bank,Dólar dos EUA,200000000.0,0,0,03/04/2024
3005,13672,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000048/2005-14,2005-12-28T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,170000000.0,1,0,19/04/2006
3077,13695,Estado,Minas Gerais,31,MG,None,Arquivado,19405.000053/2003-57,2003-08-25T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,35000000.0,0,0,20/02/2004
3217,13587,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000031/2004-78,2004-11-27T02:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,35000000.0,1,0,09/12/2004
4758,20458,Estado,Minas Gerais,31,MG,00000.000000/2020-68,Arquivado a pedido,17944.001085/2015-81,2015-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,50000000.0,0,0,29/04/2016


In [303]:
# Porém, considerando os municípios de MG, obtém-se
MG = consulta_uf_tipo('MG', 'Município')
# Verificando o status
MG[MG['status'] == 'Arquivado por decurso de prazo']['status'].value_counts()
# MG['status'].value_counts()

status
Arquivado por decurso de prazo    48
Name: count, dtype: int64

#### 3. Qual é o município da Bahia com mais solicitações deferidas?

In [305]:
# 3) Um DataFrame com os municípios da BA
mun_BA = consulta_uf_tipo('BA', 'Município')

# Fazendo uma contagem dos status por interessado
contagem = mun_BA[['interessado', 'status']]
# Contagem em relação ao status 'Deferido'
contagem_def = contagem[contagem['status'] == 'Deferido'].value_counts()
pd.DataFrame(contagem_def)

,,count
interessado,status,
Luís Eduardo Magalhães,Deferido,5
Alagoinhas,Deferido,4
Barreiras,Deferido,3
Bom Jesus da Lapa,Deferido,3
Camaçari,Deferido,2
...,...,...
Itaparica,Deferido,1
Itiruçu,Deferido,1
Alcobaça,Deferido,1


#### 4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [307]:
# 4) Façamos uso da função definida em 1.
consulta = consulta_uf_tipo('BA', 'Estado')
consulta.to_csv('Consulta_BA_Estado.csv')

In [308]:
# Lendo o arquivo .csv salvo
pd.read_csv('Consulta_BA_Estado.csv').head(3)

,Unnamed: 0,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,29,18363,Estado,Bahia,29,BA,NaN,Encaminhado à PGFN com manifestação técnica fa...,19407.000007/2005-08,2005-06-23T03:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento sustentável,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,49296000.00,1,0,25/01/2006
1,114,18503,Estado,Bahia,29,BA,NaN,Deferido,19407.000037/2002-63,2002-06-11T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5285441.39,1,0,03/07/2002
2,209,18476,Estado,Bahia,29,BA,NaN,Encaminhado à PGFN com manifestação técnica fa...,19407.000030/2005-94,2005-12-30T02:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,12000000.00,1,0,04/07/2006
